# Assignment 3: All Pairs Similarity

## Setup & Dataset Retrieval

In [1]:
import os.path
import pickle
import random

from tqdm import tqdm

import src.utilities.io as io
import src.utilities.preprocess as pp
import src.utilities.similarity_sequential as sim_seq

/Users/a/miniforge3/envs/sbruch-assignment-Lwmd/lib/python3.10/site-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
[nltk_data] Downloading package punkt to /Users/a/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/a/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/a/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
documents, data_path = io.get_data() #trec-covid as default

2023-05-27 02:24:49 - Loading Corpus...


  0%|          | 0/171332 [00:00<?, ?it/s]

2023-05-27 02:24:52 - Loaded 171332 TEST Documents.
2023-05-27 02:24:52 - Doc Example: {'text': 'OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (35%) with upper respiratory tract 

filter out the documents with less than 10 words.

In [3]:
documents = {doc_id: doc for doc_id, doc in documents.items() if len(doc['text'].split()) >= 10 and doc['text'].strip() != ''}

In [4]:
for doc_id, doc in list(documents.items())[:10]:
    num_words = len(doc["text"].split())
    print(f"Document {doc_id}: {num_words} words")

Document ug7v899j: 262 words
Document 02tnwd4m: 142 words
Document ejv2xln0: 219 words
Document 2b73a28n: 68 words
Document 9785vg6d: 110 words
Document zjufx4fo: 174 words
Document 5yhe786e: 113 words
Document 8zchiykl: 83 words
Document 8qnrcgnk: 188 words
Document jg13scgo: 189 words


### Sort the documents by their length (Longest to Shortest)

In [5]:
sort_docs = sorted(documents.items(), key=lambda x: len(x[1]['text']), reverse=True)
sorted_docs: pp.Documents = {}
for doc_id, doc in sort_docs:
    sorted_docs[doc_id] = doc

In [6]:
len(sorted_docs)

128592

In [7]:
for doc_id, doc in list(sorted_docs.items())[:50]:
    num_words = len(doc["text"].split())
    print(f"Document {doc_id}: {num_words} words")

Document ij3ncdb6: 18000 words
Document c4pt07zk: 15448 words
Document 1vimqhdp: 12745 words
Document pd1g119c: 4642 words
Document gvh0wdxn: 3507 words
Document jkrj0lbm: 3783 words
Document jkvr9lq2: 3679 words
Document y00pz4fd: 1705 words
Document lqdgvsaq: 1627 words
Document vslevssr: 1504 words
Document o1vxgzpj: 1437 words
Document bt59txxp: 1474 words
Document yt9rj42s: 1460 words
Document qapwb5e4: 1529 words
Document rff3ikak: 1490 words
Document 0prx11md: 1310 words
Document 84hh0mg9: 1312 words
Document uumzl37q: 1053 words
Document vr7vm64u: 1097 words
Document v45cb2xq: 1136 words
Document yn7pu9i8: 1204 words
Document t473tbm9: 1271 words
Document nq0qe8qf: 1151 words
Document 16b8drw2: 1163 words
Document 4cbjbo7t: 1236 words
Document tvs1snq8: 1139 words
Document 2wb007gf: 1102 words
Document st1epx6u: 1114 words
Document syl7wq15: 1114 words
Document dt3li3bk: 1054 words
Document ec798c6s: 1057 words
Document xvnxkcxq: 1032 words
Document rxvxf34z: 1039 words
Documen

In [8]:
for doc_id, doc in list(sorted_docs.items())[-50:]:
    num_words = len(doc["text"].split())
    print(f"Document {doc_id}: {num_words} words")

Document denmp8yh: 10 words
Document 1k5h0tqi: 10 words
Document 46suh53t: 10 words
Document rmlklqwo: 15 words
Document vetdsk48: 10 words
Document h29uwe1d: 11 words
Document e4t19dza: 14 words
Document kg4oljan: 10 words
Document pfxyz3q3: 10 words
Document dvlwbho2: 11 words
Document fj9le8dx: 10 words
Document bg1amnoh: 11 words
Document 60p2a1i2: 10 words
Document suqvyeac: 13 words
Document h6r6qdna: 11 words
Document 6xqmlgr2: 10 words
Document igw288oj: 15 words
Document wt8ucsjr: 10 words
Document cjw88eoq: 11 words
Document n2v9kpm2: 12 words
Document srrdh09m: 12 words
Document l9n7mklv: 11 words
Document ob9i9t04: 11 words
Document 8j184di4: 11 words
Document yfs4kbus: 10 words
Document 5rmvdtrb: 12 words
Document kxbjtyjv: 11 words
Document pcl5bn54: 12 words
Document 08grs6m7: 10 words
Document sqos28f3: 12 words
Document isn31iqt: 12 words
Document 944w4qgq: 11 words
Document 1r60beaa: 10 words
Document 3nxp0j41: 10 words
Document i9aieai9: 11 words
Document i6b2bg6a: 1

This dictionary is used to map the index of a document to the corresponding text_id in the original dataset.
This index-to-ID mappings is useful for keeping track of the original text_id of the documents.

In [9]:
doc_idx_to_id = {i: doc_id for i, doc_id in enumerate(sorted_docs.keys())}

## Due to the size, we will use different dataset samples
(1000 longest docs, 1000 shortest docs, 1000 random docs)

### Define the paths to save and read the samples and their mappings

In [11]:
sample_size = 100
samples_dir = os.path.join(data_path, "samples_" + str(sample_size))
if not os.path.exists(samples_dir):
    os.mkdir(samples_dir)

# longest_docs_path = os.path.join(samples_dir, "longest_docs.jsonl")
# shortest_docs_path = os.path.join(samples_dir, "shortest_docs.jsonl")
# random_docs_path = os.path.join(samples_dir, "random_docs.jsonl")
random_docs_idx_path = os.path.join(samples_dir, "random_docs_idx.pkl")

In [12]:
if not os.path.exists(random_docs_idx_path):
# if not all([os.path.exists(p) for p in [longest_docs_path, shortest_docs_path, random_docs_path, random_docs_idx_path]]):
    # longest_docs = pp.Documents = {}
    # for doc_id, doc in list(sorted_docs.items())[:sample_size]:
    #     longest_docs[doc_id] = doc
    # longest_doc_idx_to_id = {i:doc_idx_to_id[idx] for i, idx in enumerate(range(sample_size))}
    #
    # shortest_docs = pp.Documents = {}
    # for doc_id, doc in list(sorted_docs.items())[-sample_size:]:
    #     shortest_docs[doc_id] = doc
    # shortest_doc_idx_to_id = {i:doc_idx_to_id[idx] for i, idx in enumerate(range(len(sorted_docs)-sample_size, len(sorted_docs)))}
    #
    # random_docs = pp.Documents = {}
    # s_docs = list(sorted_docs.items())
    random_docs_idx = random.sample(range(len(sorted_docs)), k=sample_size)
    # for index in random_docs_idx:
    #     doc_id, doc = s_docs[index]
    #     random_docs[doc_id] = doc
    # random_doc_idx_to_id = {i:doc_idx_to_id[idx] for i, idx in enumerate(random_docs_idx)}

    # io.save_sample_data(longest_docs_path, longest_docs)
    # io.save_sample_data(shortest_docs_path, shortest_docs)
    # io.save_sample_data(random_docs_path, random_docs)

    with open(random_docs_idx_path, "wb") as f:
        pickle.dump(random_docs_idx, f)
else:
    # longest_docs = io.get_sample_data(longest_docs_path)
    # shortest_docs = io.get_sample_data(shortest_docs_path)
    # random_docs = io.get_sample_data(random_docs_path)
    with open(random_docs_idx_path, "rb") as f:
        random_docs_idx = pickle.load(f)

longest_docs = pp.Documents = {}
for doc_id, doc in list(sorted_docs.items())[:sample_size]:
    longest_docs[doc_id] = doc
longest_doc_idx_to_id = {i:doc_idx_to_id[idx] for i, idx in enumerate(range(sample_size))}

shortest_docs = pp.Documents = {}
for doc_id, doc in list(sorted_docs.items())[-sample_size:]:
    shortest_docs[doc_id] = doc
shortest_doc_idx_to_id = {i:doc_idx_to_id[idx] for i, idx in enumerate(range(len(sorted_docs)-sample_size, len(sorted_docs)))}

random_docs = pp.Documents = {}
s_docs = list(sorted_docs.items())
for index in random_docs_idx:
    doc_id, doc = s_docs[index]
    random_docs[doc_id] = doc
random_doc_idx_to_id = {i:doc_idx_to_id[idx] for i, idx in enumerate(random_docs_idx)}

Mapping is useful if we want to retrieve the original documents corresponding to the randomly chosen documents in random_docs

In [13]:
samples = [("longest_docs", longest_docs, longest_doc_idx_to_id),
           ("shortest_docs", shortest_docs, shortest_doc_idx_to_id),
           ("random_docs", random_docs, random_doc_idx_to_id)]

In [14]:
# temp for quello di lotto
samples_tmp = [("longest_docs", list(longest_docs.keys())),
                ("shortest_docs", list(shortest_docs.keys())),
                 ("random_docs", list(random_docs.keys()))]

## Tokenization
Document cleaning and tokenization

In [15]:
tokens_dir = os.path.join(samples_dir, "tokens")
if not os.path.exists(tokens_dir):
    os.mkdir(tokens_dir)

docs_tokens_path = os.path.join(tokens_dir, "doc_tokens.pkl")

In [16]:
if not os.path.exists(docs_tokens_path):
    tokenized_samples = []
    for name, docs, idx_to_id in samples:
        tokenized_docs: pp.TokenizedDocuments = pp.get_tokenized_documents(docs)
        tokenized_samples.append((name, tokenized_docs, idx_to_id))

    with open(docs_tokens_path, "wb") as f:
        pickle.dump(tokenized_samples, f)
else:
    with open(docs_tokens_path, "rb") as f:
        tokenized_samples = pickle.load(f)

Tokenizing documents: 100%|██████████| 100/100 [00:00<00:00, 241.21it/s]


## VECTORIZE

In [17]:
tfidf_dir = os.path.join(samples_dir, "tfidf")
if not os.path.exists(tfidf_dir):
    os.mkdir(tfidf_dir)

tfidf_results_path = os.path.join(tfidf_dir, "tfidf_docs.pkl")

In [18]:
if not os.path.exists(tfidf_results_path):
    tfidf_docs = []
    for name, docs, idx_to_id in tqdm(tokenized_samples):
        vectorized_docs, time_taken = pp.vectorize(docs)
        tfidf_docs.append((name, vectorized_docs, time_taken, idx_to_id))

    with open(tfidf_results_path, "wb") as f:
        pickle.dump(tfidf_docs, f)
else:
    with open(tfidf_results_path, "rb") as f:
        tfidf_docs = pickle.load(f)

100%|██████████| 3/3 [00:00<00:00, 14.07it/s]


In [25]:
for name, _, time_taken, _ in tfidf_docs:
    print(f"Time taken to vectorize {name}: {time_taken:.6f} seconds")

Time taken to vectorize longest_docs: 0.166941 seconds
Time taken to vectorize shortest_docs: 0.003401 seconds
Time taken to vectorize random_docs: 0.023076 seconds


## Sequential APDS

In [31]:
thresholds = [0.1, 0.3, 0.5, 0.7, 0.9] # Chosen thresholds

In [32]:
seq_results_dir = os.path.join(samples_dir, "seq_result")
if not os.path.exists(seq_results_dir):
    os.mkdir(seq_results_dir)

seq_results_path = os.path.join(seq_results_dir, "seq_results.pkl")

In [33]:
all_seq_results = {} #sample: pairs indices, pairs docs id, additional information

if not os.path.exists(seq_results_path):
    for name, vectorized_docs, tfidf_time, idx_to_id in tqdm(tfidf_docs):
        seq_results = []
        for threshold in thresholds:
            similar_pairs, information = sim_seq.sequential_APDS(vectorized_docs, name, threshold, tfidf_time)
            mapped_pairs = sim_seq.map_doc_idx_to_id(similar_pairs, idx_to_id)
            seq_results.append((similar_pairs, mapped_pairs, information))
        all_seq_results[name] = seq_results

    with open(seq_results_path, "wb") as f:
        pickle.dump(all_seq_results, f)
else:
    with open(seq_results_path, "rb") as f:
        all_seq_results = pickle.load(f)

100%|██████████| 3/3 [00:00<00:00, 49.56it/s]


### longest 1000

In [34]:
sim_seq.print_results(all_seq_results, "longest_docs")

--Run info--
sample_name: longest_docs
threshold: 0.1
pairs_count: 1213
tfidf_time: 0.16694116592407227
cosine_time: 0.007980108261108398
find_time: 0.002804994583129883
total_time: 0.17772626876831055

--Run info--
sample_name: longest_docs
threshold: 0.3
pairs_count: 60
tfidf_time: 0.16694116592407227
cosine_time: 0.004389047622680664
find_time: 0.0011210441589355469
total_time: 0.17245125770568848

--Run info--
sample_name: longest_docs
threshold: 0.5
pairs_count: 40
tfidf_time: 0.16694116592407227
cosine_time: 0.004350900650024414
find_time: 0.0010399818420410156
total_time: 0.1723320484161377

--Run info--
sample_name: longest_docs
threshold: 0.7
pairs_count: 34
tfidf_time: 0.16694116592407227
cosine_time: 0.005455970764160156
find_time: 0.0015180110931396484
total_time: 0.17391514778137207

--Run info--
sample_name: longest_docs
threshold: 0.9
pairs_count: 27
tfidf_time: 0.16694116592407227
cosine_time: 0.005494117736816406
find_time: 0.0008308887481689453
total_time: 0.173266172

In [35]:
sim_seq.print_first_pair(all_seq_results, "longest_docs", tokenized_samples)

ij3ncdb6 and c4pt07zk have 0.3055 similarity

ij3ncdb6:
xxiv world allergy congress 2015 seoul korea 14-17 october 2015 a1 pirfenidone inhibits tgf-b1-induced extracellular matrix production nasal polyp-derived fibroblast jae-min shin heung-man lee il-ho park a2 efficacy 2-week course oral steroid treatment chronic spontaneous urticaria refractory antihistamine hyun-sun yoon gyeong yul park a3 altered distribution follicular helper cell may predict pronounced clinical course primary sjögren ’ syndrome margit zeher a4 betamethasone suppresses th2 cell development induced langerhans cell like dendritic cell katsuhiko matsui saki tamai reiko ikeda a5 evaluation variousallergens case allergic bronchial asthma lucknow neighbouring district intradermal skintest drsushil suri dranu suri a6 evaluation ferqency adhd childhood asthma marzieh heidarzadeh arani a7 steven johnson syndrome caused typhoid fever child azwin lubis anang endaryanto a8 chronic bronchitis radio contrast medium hypersensit

### shortest 1000

In [36]:
sim_seq.print_results(all_seq_results, "shortest_docs")

--Run info--
sample_name: shortest_docs
threshold: 0.1
pairs_count: 234
tfidf_time: 0.0034008026123046875
cosine_time: 0.0007009506225585938
find_time: 0.0008339881896972656
total_time: 0.004935741424560547

--Run info--
sample_name: shortest_docs
threshold: 0.3
pairs_count: 20
tfidf_time: 0.0034008026123046875
cosine_time: 0.0005638599395751953
find_time: 0.0008282661437988281
total_time: 0.004792928695678711

--Run info--
sample_name: shortest_docs
threshold: 0.5
pairs_count: 12
tfidf_time: 0.0034008026123046875
cosine_time: 0.0013430118560791016
find_time: 0.0017070770263671875
total_time: 0.0064508914947509766

--Run info--
sample_name: shortest_docs
threshold: 0.7
pairs_count: 12
tfidf_time: 0.0034008026123046875
cosine_time: 0.002260923385620117
find_time: 0.0008080005645751953
total_time: 0.0064697265625

--Run info--
sample_name: shortest_docs
threshold: 0.9
pairs_count: 11
tfidf_time: 0.0034008026123046875
cosine_time: 0.0008027553558349609
find_time: 0.0007922649383544922
tot

In [37]:
sim_seq.print_first_pair(all_seq_results, "shortest_docs", tokenized_samples)

a37d70ec and 5izfdx4y have 0.1329 similarity

a37d70ec:
finding camel-ot holy grail pandemic sars-cov-2 engineered camelid antibody multimers potently block sars-cov-2 viral entry

5izfdx4y:
sniffing covid-19 georgina mill report dog trained detect coronavirus

zf19gppx and agnmfkoq have 1.0000 similarity

zf19gppx:
saliva non-invasive specimen detection sars-cov-2 diagnostic testing covid-19 central controlling global pandemic

agnmfkoq:
u equity risk premium covid-19 pandemic study equity risk premium united state covid-19 pandemic

zf19gppx and agnmfkoq have 1.0000 similarity

zf19gppx:
saliva non-invasive specimen detection sars-cov-2 diagnostic testing covid-19 central controlling global pandemic

agnmfkoq:
u equity risk premium covid-19 pandemic study equity risk premium united state covid-19 pandemic

zf19gppx and agnmfkoq have 1.0000 similarity

zf19gppx:
saliva non-invasive specimen detection sars-cov-2 diagnostic testing covid-19 central controlling global pandemic

agnmfkoq:

### random 1000

In [38]:
sim_seq.print_results(all_seq_results, "random_docs")

--Run info--
sample_name: random_docs
threshold: 0.1
pairs_count: 233
tfidf_time: 0.02307605743408203
cosine_time: 0.0025048255920410156
find_time: 0.0020737648010253906
total_time: 0.027654647827148438

--Run info--
sample_name: random_docs
threshold: 0.3
pairs_count: 4
tfidf_time: 0.02307605743408203
cosine_time: 0.0012917518615722656
find_time: 0.0008111000061035156
total_time: 0.025178909301757812

--Run info--
sample_name: random_docs
threshold: 0.5
pairs_count: 0
tfidf_time: 0.02307605743408203
cosine_time: 0.0010907649993896484
find_time: 0.0010259151458740234
total_time: 0.025192737579345703

--Run info--
sample_name: random_docs
threshold: 0.7
pairs_count: 0
tfidf_time: 0.02307605743408203
cosine_time: 0.0013740062713623047
find_time: 0.000949859619140625
total_time: 0.02539992332458496

--Run info--
sample_name: random_docs
threshold: 0.9
pairs_count: 0
tfidf_time: 0.02307605743408203
cosine_time: 0.0012280941009521484
find_time: 0.00102996826171875
total_time: 0.025334119796

In [39]:
sim_seq.print_first_pair(all_seq_results, "random_docs", tokenized_samples)

h29uwe1d and oufi7w1d have 0.1038 similarity

h29uwe1d:
avian influenza aided readiness swine flu despite gain threat bird flu pandemic preparedness patchy

oufi7w1d:
wet markets—a continuing source severe acute respiratory syndrome influenza summary context live-animal market wet market provide source vertebrate invertebrate animal customer tropical subtropical region world wet market sell live poultry fish reptile mammal every kind live-poultry market mostly chicken pigeon quail duck goose wide range exotic wild-caught farm-raised fowl usually separated market selling fish red-meat animal stall near physical separation despite widespread availability affordable refrigeration many asian people prefer live animal fresh produce wet market widespread asian country country asian people migrated live-poultry market source h5n1 bird-influenza virus transmitted killed six 18people hong kong starting point yi guan colleague science 2003 302 276–78 recently reported isolation severe acute resp

## Parallel APDS

# MIRACLE PARALLEL?

In [12]:
from typing import Dict, List, Tuple
import csv

In [13]:
def create_doc_sim_csv(pairs_list: List[Tuple[str, str, float]], ds_name: str,
                       threshold: float, type: str | None, workers: None | int = None ) -> None:
    '''
    PURPOSE: create the .csv file sotring the list of similar documents pairs with the cosine similarity
    ARGUMENTS:
        - pairs_list (List[Tuple[str, str, float]]): list of unique similar pair with the similarity
        - ds_name: (str): dataset name
        - threshold (float): used threshold
        - type (str | None): type of sequential version
        - workers (None | int): number of workers used
    RETURN: None
    '''

    path = ''
    if type is not None:
        if not os.path.exists(f'./results/{ds_name}/{threshold}'): os.makedirs(f'./results/{ds_name}/{threshold}')
        path = f'./results/{ds_name}/{threshold}/{type}_sequential.csv'
    else:
        if not os.path.exists(f'./results/{ds_name}/{threshold}/pyspark/'): os.makedirs(f'./results/{ds_name}/{threshold}/pyspark/')
        path = f'./results/{ds_name}/{threshold}/pyspark/{workers}_workers.csv'
    if not os.path.exists(path): # If there is already a file, return
        with open(path, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerows(pairs_list)

In [14]:
import itertools
from src.utilities.similarity_mapreduce_spark import pyspark_APDS
import pandas as pd
from typing import Tuple, Type, List, Dict, Any

In [ ]:
# datasets = ['nfcorpus'] # Choosen datasets
# thresholds: List[float] = [0.5, 0.6, 0.7, 0.8, 0.9] # Choosen thresholds
# numslices_factor = [1, 2, 5, 10, 15] # Choosen numslices factors N_PARTITIONS
# max_workers = [1, 2, 5, 10, 15] # n_executors
# considered_docs = 750

# # Download datasets
# datasets_data = {dataset: download_dataset(dataset) for dataset in datasets}
#
# # Pre-process and sample with the original datasets
# pre_processed_data = {dataset: sample_dict(documents_preprocessing(dataset, docs_dict), considered_docs)
#                       for dataset, docs_dict in datasets_data.items()}

In [ ]:
# for collection_name, collection_docs, _ in samples:
pyspark_results = []

thresholds: List[float] = [0.001] # Choosen thresholds
numslices_factor = [5] # Choosen numslices factors N_PARTITIONS
max_workers = 5# n_executors

for ds_name, sampled_dict, _ in samples:
# for ds_name, sampled_dict in pre_processed_data.items():

    print(f'\n------------ALL PAIRS DOCUMENTS SIMILARITY - {ds_name}------------')

    for threshold in thresholds:
        print(f'\n--------Running with threshold: {threshold}--------')

        for s_factor, workers in itertools.product(numslices_factor, range(1, max_workers + 1)):
            # PySpark Execution

            print(f'\nPySpark Parallel Execution with {workers} workers and slice factor of {s_factor}')
            sim_doc_ps, ps_res = pyspark_APDS(ds_name=ds_name, sampled_dict=sampled_dict, threshold=threshold, workers=workers, s_factor=s_factor)
            pyspark_results.append(ps_res)
            create_doc_sim_csv(sim_doc_ps, ds_name, threshold, None, workers)
            print(' Done')

        print('\n')


In [20]:

print('\nSaving pyspark_results')
dio = pd.DataFrame.from_dict(
    dict(zip(range(len(pyspark_results)), pyspark_results)),
    orient='index',
    columns=[
        'ds_name',
        'elapsed',
        'threshold',
        'unique_pairs_sim_docs',
        'workers',
        'slice_factor'
    ],
)
dio.to_csv('./results/pyspark_results.csv', index=False)
print( 'Done')


Saving pyspark_results
Done


In [ ]:
dio

In [ ]:
pyspark_results

MAGICLOTTO?

In [30]:
# # from utility import data_preparation
#
# # spark
# import findspark
#
# findspark.init()
#
# from pyspark.conf import SparkConf
# from pyspark.sql import SparkSession
#
# # spark algorithm
# import itertools
# import numpy as np
#
# # test
# import time
# import multiprocessing
# import pandas as pd
# import os
#
# # typing
# from typing import List, Tuple, Any
# from scipy.sparse import csr_matrix

In [32]:
# def spark_(doc_matrix: np.array, keys: list, threshold: float, n_workers: int = 8, n_slices: int = 1) -> Tuple[List[Tuple[tuple, float]], float]:
#     """
#     Spark "Documents All Pairs Similarity" algorithm.
#
#     Args:
#         doc_matrix (np.array): document matrix in numpy format.
#         keys (list): list of document ids.
#         threshold (float): threshold.
#         n_workers (int, optional): number of worker to be feed to spark. Defaults to 8.
#         n_slices (int, optional): number of data divisions for each worker. Defaults to 1.
#
#     Returns:
#         Tuple[ List[Tuple[tuple,float]], float ]:
#             -documents id pairs along with their similarity (List[Tuple[tuple,float]])
#             -execution time (float)
#     """
#
#     # Create SparkSession
#     spark = SparkSession\
#         .builder\
#         .config(conf=SparkConf().setMaster(f"local[{n_workers}]")
#                 .setAppName("all_pairs_docs_similarity.com")
#                 .set("spark.executor.memory", "10g")
#                 .set("spark.executor.cores", "1")
#                 .set("spark.driver.memory", "10g"))\
#         .getOrCreate()
#
#     # Get sparkContextxt
#     sc = spark.sparkContext
#
#     # sort the document matrix by maximal term frequency in the entire corpus.
#     term_freq = np.sum(doc_matrix > 0, axis=0)
#     sorted_terms_indexes = np.argsort(term_freq)[::-1]
#     doc_matrix = np.array([row[sorted_terms_indexes] for row in doc_matrix])
#
#     start_spark = time.perf_counter()
#
#     # 1) Zip each document id with its vectorial representation
#     keys_rdd = sc.parallelize(keys, n_workers * n_slices)
#     vectorized_docs_rdd = keys_rdd.zip(sc.parallelize(doc_matrix, n_workers * n_slices)).persist()
#
#     # 2) broadcast variables to be used by spark
#
#     def compute_d_star(doc1: np.array, doc2: np.array) -> np.array:
#         """
#         Function to compute d_star with spark.
#
#         Args:
#             doc1 (np.array)
#             doc2 (np.array)
#
#         Returns:
#             np.array: array of element-wise maximum values of doc1 and doc2.
#         """
#
#         return np.maximum(doc1, doc2)
#
#     def non_zero_terms(doc: np.array) -> np.array:
#         """
#         Function to find the non-zero element indexes for the given document with spark.
#
#         Args:
#             doc (np.array)
#
#         Returns:
#             np.array: array of indexes.
#         """
#
#         return np.nonzero(doc)[0]
#
#     threshold_broadcast = sc.broadcast(threshold)
#
#     # d_star broadcast computed by spark
#     d_star_broadcast = sc.broadcast(vectorized_docs_rdd.values().reduce(compute_d_star))
#
#     # vectorized document broadcast
#     vectorized_docs_broadcast = sc.broadcast(dict(vectorized_docs_rdd.collect()))
#
#     # non-zero term ids broadcast
#     doc_id_set_term_broadcast = sc.broadcast(dict(vectorized_docs_rdd.mapValues(non_zero_terms).collect()))
#
#     def MAP(doc_pair: Tuple[Any, np.array]) -> List[Tuple[int, Any]]:
#         """
#         MAP function for the "Documents All Pairs Similarity" algorithm.
#
#         Args:
#             doc_pair (Tuple[Any,np.array]):
#                 -document id (Any).
#                 -vectorized doc (np.array).
#
#         Returns:
#             List[Tuple[int,Any]]: list of (term id,doc_id)
#         """
#
#         # unpack document pair
#         id, doc = doc_pair
#
#         # loop variables
#         dot_prod = 0
#         term = 0
#
#         # b(d) computation: at the end of this while loop b(d) = term-1.
#         while (dot_prod < threshold_broadcast.value):
#
#             # if there is no more terms then a empty list is returned:
#             # this document is excluded.
#             if term >= doc.shape[0]:
#                 return []
#
#             dot_prod += ((doc[term]) * (d_star_broadcast.value[term]))
#
#             term += 1
#
#         term__doc_ids = []
#
#         # if the term has TF-IDF=0 then it can not be part of an intersection, thus it is excluded.
#         non_zeros = np.nonzero(doc)[0]
#
#         # take only the term which are >= (term-1) (b(d)).
#         non_zeros = non_zeros[non_zeros >= (term - 1)]
#
#         # append for each term (term,id).
#         for term in non_zeros:
#             term__doc_ids.append((term, id))
#
#         return term__doc_ids
#
#     def filter_pairs(pair: Tuple[int, list]) -> List[Tuple[Any, Any]]:
#         """
#         First part of REDUCE function for the "Documents All Pairs Similarity" algorithm.
#         Function to generate (doc_id, doc_id) pairs on which the similarity will be computed.
#
#         Args:
#             pair (Tuple[int,list]):
#                 -term id.
#                 -list of document ids.
#
#         Returns:
#             List[Tuple[Any,Any]]: list of (doc_id, doc_id) on which the similarity will be computed.
#         """
#
#         # unpack term_id, id_list.
#         term_id, id_list = pair
#
#         pairs = []
#
#         # iterate over combinations of document ids: this ensures that the document ids pair is unique.
#         for id1, id2 in itertools.combinations(id_list, 2):
#
#             # set intersection for the non-zeros term of the two documents.
#             common_terms = np.intersect1d(doc_id_set_term_broadcast.value[id1],
#                                           doc_id_set_term_broadcast.value[id2],
#                                           assume_unique=True)
#
#             # if the max of the intersection is equal to the tem_id then append.
#             if max(common_terms) == term_id:
#                 pairs.append((id1, id2))
#
#         return pairs
#
#     def compute_similarity(pair: Tuple[Any, Any]) -> Tuple[Tuple[Any, Any], float]:
#         """
#         Second part of REDUCE function for the "Documents All Pairs Similarity" algorithm. Similarity computation function.
#
#         Args:
#             pair (Tuple[Any,Any]): doc_ids pair.
#
#         Returns:
#             Tuple[Tuple[Any,Any],float]:
#                 -(doc_id, doc_id) (Tuple[Any,Any])
#                 -similarity (float)
#         """
#
#         id1, id2 = pair
#
#         similarity = np.dot(vectorized_docs_broadcast.value[id1], vectorized_docs_broadcast.value[id2].transpose())
#
#         return ((id1, id2), similarity)
#
#     def similar_doc(pair: Tuple[Tuple[Any, Any], float]) -> bool:
#         """
#         Third part of REDUCE function for the "Documents All Pairs Similarity" algorithm.
#         This function excludes pairs which similarity is < threshold.
#
#         Args:
#             pair (Tuple[Tuple[Any,Any],float]):
#                     -(doc_id, doc_id) (Tuple[Any,Any])
#                     -similarity (float)
#
#         Returns:
#             bool:
#                 -True: take the pair.
#                 -False: remove the pair.
#         """
#
#         _, similarity = pair
#
#         return similarity >= threshold_broadcast.value
#
#     # 3) Compute with spark:
#     #   1. MAP function using flatMap(MAP).
#     #   2. Group by term id.
#     #   3. REDUCE function using:
#     #       1. flatMap(filter_pairs)
#     #       2. map(compute_similarity)
#     #       3. filter(similar_doc)
#     similarity_doc_pairs = vectorized_docs_rdd\
#         .flatMap(MAP)\
#         .groupByKey()\
#         .flatMap(filter_pairs)\
#         .map(compute_similarity)\
#         .filter(similar_doc)
#
#     to_return = similarity_doc_pairs.collect()
#
#     end_spark = time.perf_counter()
#
#     spark.stop() # Stop spark session
#
#     return to_return, (end_spark - start_spark)

In [36]:
# def comparison(keys: list, doc_matrix: csr_matrix, threshold: float, n_workers: int, n_slices: int):  # -> Tuple[float, float, set]:
#     """_summary_
#
#     Args:
#         keys (list): list of document keys.
#         doc_matrix (csr_matrix): document matrix in csr format.
#         threshold (float): threshold.
#         n_workers (int):  number of worker to be feed to spark.
#         n_slices (int): number of data divisions for each worker.
#
#     Returns:
#         Tuple[float,float,set]:
#             -spark execution time (float).
#             -seq execution time (float).
#             -set of missing spark document id pairs. (set)
#     """
#
#     # spark computation
#     spark_list, spark_elapsed = spark_(doc_matrix.toarray(), keys, threshold, n_workers, n_slices)
#
#     # seq computation
#     # seq_list, seq_elapsed = sequential(doc_matrix, keys, threshold)
#
#     # print("\nspark result len:", len(spark_list), "seq result len:", len(seq_list))
#
#     # print missing spark doc_id pairs
#     # missing_spark = set(dict(seq_list).keys()) - set(dict(spark_list).keys())
#     # print("spark missing pairs: ", missing_spark, len(missing_spark))
#
#     # print time
#     print("\nspark time: ", spark_elapsed)
#     # print("seq time: ", seq_elapsed)
#
#     return spark_elapsed #, seq_elapsed, missing_spark

In [ ]:
# dataset = "nfcorpus"
#
# data = {
#     "threshold": [],
#     "n_workers": [],
#     "n_slices": [],
#     "spark_time": [],
#     "seq_time": [],
#     "missing": []
# }
#
# # samples = [("longest_docs", longest_docs, longest_doc_idx_to_id),
# #            ("shortest_docs", shortest_docs, shortest_doc_idx_to_id),
# #            ("random_docs", random_docs, random_doc_idx_to_id)]
# # samples_tmp = [("longest_docs", list(longest_docs.keys())),
# #                ("shortest_docs", list(shortest_docs.keys())),
# #                ("random_docs", list(random_docs.keys()))]
#
# # keys, doc_matrix = data_preparation(dataset, 1000)
# keys = list(shortest_docs.keys())
# doc_matrix = shortest_docs
#
# for threshold in [0.2]: #, 0.4, 0.6]:
#
#     for n_workers in [1, 2, 4]: #, 6, 8, 12, 16]:
#
#         for n_slices in [4]: #[1, 3, 9]:
#             print("\n\nTEST FOR:")
#             print("threshold: ", threshold)
#             print("n_workers: ", n_workers)
#             print("n_slices: ", n_slices, "\n")
#
#             spark_time, seq_time, missing = comparison(keys, doc_matrix, threshold, n_workers,n_slices)
#
#             data["threshold"].append(threshold)
#             data["n_workers"].append(n_workers)
#             data["n_slices"].append(n_slices)
#             data["spark_time"].append(spark_time)
#             data["seq_time"].append(seq_time)
#             data["missing"].append(missing)
#
# data = pd.DataFrame.from_dict(data)
# data.to_parquet("data.parquet")